# Following below youtube tutorial

### Youtube video : https://www.youtube.com/watch?v=QIUxPv5PJOY&ab_channel=ComputerScience

### Going to analyse for indian stock : TATA MOTORS

* It is mentioned in the medium article : https://randerson112358.medium.com/stock-price-prediction-using-python-machine-learning-e82a039ac2bb that it uses Long Short-Term Memory (LSTM)


In [4]:
#Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`